In [62]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm, invgamma
import pickle
import time
import math

### Load temperatures data

In [2]:
df_temp=pd.read_csv('data/temp_2016.csv',sep=',')
df_temp=df_temp.drop(df_temp.columns[0],axis=1)
df_temp=df_temp.sort_values(by='date')

In [3]:
#temperatures every day at 3pm
temp_day=df_temp[df_temp.hour=="15:00"]
temp_day.head()
T_h=list(temp_day.t)

### Load electricity data

In [4]:
mypath="data/"
df=pd.read_csv(mypath+"cdc_conso_daytypes.csv",sep=",",encoding='latin-1')
df.index=range(0,len(df))

In [5]:
df=df.sort_values(by=['Date'])
df.index=range(0,len(df))
#Valeurs intermédiaires à partir de 2017

In [6]:
#On se concentre d'abord sur une année afin d'initialiser le filtre
df=df[(df.Date >'2016-01-01') & (df.Date <'2017-01-01')]
df.index=range(0,len(df))

# Gibbs Sampling

## Parameters required

In [33]:
nb_days = 15
temperatures = T_h[:nb_days] #france mean temp in jan 2016
daytypes = list(df[df['Heure']=='15:00']['daytype'][:nb_days])
consumptions = list(df[df['Heure']=='15:00']['Consommation (MW)'][:nb_days])
nb_particles = 10**5

sigma2 = invgamma.rvs(a=10**(-2), scale=10**(2), size = 1)[0]

N_daytype = 9
k_day = np.random.dirichlet(np.ones(N_daytype),1)
kappa = k_day[0].tolist()

u_heat = np.random.normal(14,1)  #fixed parameter, invariant to time

13.624020667916607

In [34]:
%load_ext autoreload
%autoreload 2
from elec_forecast.bootstrap_filter_model import BootstrapFilterModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
#Data from last year's group
u_heat = 7.3
kappa = np.array([1/9]*9)
sigma2 = 27000000
sigma2_s_param = 79000000
sigma2_g_param = 127000000
nb_particles = 10000
#len_filtering = 1000

In [56]:
#Execute Gibbs sampling
BFM = BootstrapFilterModel(temperatures,daytypes,consumptions,nb_days,nb_particles,sigma2,kappa,u_heat)

In [57]:
start_time = time.time()
BFM.bf_initialization_gibbs()
print("--- %s seconds ---" % (time.time() - start_time))

Gibbs sampling for particle 1/10000
Gibbs sampling for particle 1000/10000
Gibbs sampling for particle 2000/10000
Gibbs sampling for particle 3000/10000
Gibbs sampling for particle 4000/10000
Gibbs sampling for particle 5000/10000
Gibbs sampling for particle 6000/10000
Gibbs sampling for particle 7000/10000
Gibbs sampling for particle 8000/10000
Gibbs sampling for particle 9000/10000
--- 35.17565727233887 seconds ---


In [26]:
variables = BFM.__dict__.keys()
variables

dict_keys(['temperatures', 'daytypes', 'consumptions', 'nb_days', 'nb_particles', 'sigma2', 'kappa', 'u_heat', 's', 'g_heat', 'sigma_s_star_2', 'sigma_g_star_2', 'x_season', 'x_heat', 'x', 'w'])

In [60]:
BFM.x

array([  2.82190226e+07,   1.93865926e+06,   6.21578137e+05, ...,
         1.60501798e-07,   6.44060718e-08,   9.09615775e-08])

In [61]:
sum(BFM.w)

15.114924235721189

In [64]:
#Write initial parameters from Gibbs sampling in a file
timestr = time.strftime("%Y%m%d-%H%M%S")

output_file ='data/parameters_init_' + timestr + '.pkl'
parameters = {
    's_init':BFM.s[nb_days-1,],
    'g_heat_init':BFM.g_heat[nb_days-1,:],
    'sigma_s_init':np.sqrt(BFM.sigma_s_star_2[0,nb_particles-1]),
    'sigma_g_init':np.sqrt(BFM.sigma_g_star_2[0,nb_particles-1]),
    'x_init':BFM.x,
    'w_init':BFM.w,
    'sigma2': sigma2,
    'u_h':u_heat,
    'kappa': kappa
}
output = open(output_file, 'wb')
pickle.dump(parameters, output)
output.close()